In [5]:
import pandas as pd
import numpy as np
import os
from bokeh.charts import Scatter, output_file, show, BoxPlot, Histogram
from bokeh.sampledata.autompg import autompg as bdf
from bokeh.charts import defaults
import matplotlib

defaults.width = 700
defaults.height = 600

PROJECT_ROOT = os.path.dirname(os.path.abspath('__file__'))
DATA_PATH = os.path.join(PROJECT_ROOT,'all_users_gender_score.csv')

In [6]:
df = pd.read_csv(DATA_PATH)

In [7]:
#summary table for all users
df.drop(['id'],axis=1).describe()

,messages_sent,msg_scored
count,240131.000000,165870.000000
mean,6.505333,0.061019
std,13.097884,0.219752
min,0.000000,-1.000000
25%,0.000000,0.000000
50%,2.000000,0.000000
75%,8.000000,0.165714
max,1037.000000,1.000000


In [8]:
# summary table for female/male 
df.drop(['id'],axis=1).groupby(['gender'],as_index=True).describe()

messages_sent    msg_scored
gender                                   
F      count  118706.000000  93312.000000
       mean        7.671769      0.061262
       std        13.837930      0.220725
       min         0.000000     -1.000000
       25%         2.000000      0.000000
       50%         2.000000      0.000000
       75%         8.000000      0.166667
       max      1037.000000      1.000000
M      count  105338.000000  72373.000000
       mean        6.177809      0.060728
       std        12.928883      0.218492
       min         0.000000     -1.000000
       25%         0.000000      0.000000
       50%         2.000000      0.000000
       75%         6.000000      0.162500
       max       771.000000      1.000000

In [9]:
#covariance by gender  
grouped = df.groupby('gender')
grouped.apply(lambda x: x['messages_sent'].cov(x['msg_scored']))

gender
F   -0.006392
M    0.003763
dtype: float64

In [11]:
# total messages sent
df['messages_sent'].sum()

1562132.0

In [13]:
# total messages sent per user by gender
grouped = df.groupby('gender')
print(grouped)
data = grouped['messages_sent'].sum()
print(data)

gender
F    910685.0
M    650758.0
Name: messages_sent, dtype: float64
